# Hackman Hackathon
_____________________________________________________________________________________________________________________

### "The Oracle": Using HMM with a trigraph and a dropoff (fallback) feature to a bigraph and unigraph analysis. 
### "The Brains": A simple Greedy RL agent that picks the letter with the highest probability i.e, the letter which reduces the solution space the most. 
_____________________________________________________________________________________________________________________

## Importing the required Libraries, Loading the corpus and test datasets

In [2]:
import numpy as np
import collections
import os 

ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
CHAR_TO_INDEX = {char: i for i, char in enumerate(ALPHABET)}
INDEX_TO_CHAR = {i: char for i, char in enumerate(ALPHABET)}

print("Libraries imported and alphabet defined.")

Libraries imported and alphabet defined.


In [3]:

KAGGLE_CORPUS_PATH = "/kaggle/input/corpus2/corpus.txt"

def load_corpus(filename=KAGGLE_CORPUS_PATH):
    """
    Reads the corpus file from the specified Kaggle path
    and groups words by length.
    """
    words_by_length = collections.defaultdict(list)
    
    if not os.path.exists(filename):
        print(f"--- ERROR ---")
        print(f"File not found at: {filename}")
        print("Please verify the 'corpus' folder name and 'corpus.txt' file name in your Kaggle input.")
        
        # Helper to debug: list contents of the input directory
        print("\nListing contents of /kaggle/input/ ...")
        try:
            input_dirs = os.listdir("/kaggle/input")
            print(f"Found: {input_dirs}")
            
            # If the 'corpus' directory exists, list its contents
            if "corpus" in input_dirs:
                print("\nListing contents of /kaggle/input/corpus/ ...")
                print(os.listdir("/kaggle/input/corpus"))
                
        except FileNotFoundError:
            print("Could not find /kaggle/input/ directory.")
        except Exception as e:
            print(f"An error occurred while listing directories: {e}")
        return None

    # File exists, proceed to load
    try:
        with open(filename, 'r') as f:
            for word in f:
                cleaned_word = word.strip().upper()
                if cleaned_word: # Ensure it's not an empty line
                    words_by_length[len(cleaned_word)].append(cleaned_word)
        
        print(f"Corpus loaded successfully from {filename}.")
        print("\nWord counts by length:")
        for length in sorted(words_by_length.keys()):
            print(f"  Length {length}: {len(words_by_length[length])} words")
        return words_by_length
        
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

words_by_length = load_corpus()

Corpus loaded successfully from /kaggle/input/corpus2/corpus.txt.

Word counts by length:
  Length 1: 46 words
  Length 2: 84 words
  Length 3: 388 words
  Length 4: 1169 words
  Length 5: 2340 words
  Length 6: 3755 words
  Length 7: 5111 words
  Length 8: 6348 words
  Length 9: 6808 words
  Length 10: 6465 words
  Length 11: 5452 words
  Length 12: 4292 words
  Length 13: 3094 words
  Length 14: 2019 words
  Length 15: 1226 words
  Length 16: 698 words
  Length 17: 375 words
  Length 18: 174 words
  Length 19: 88 words
  Length 20: 40 words
  Length 21: 16 words
  Length 22: 8 words
  Length 23: 3 words
  Length 24: 1 words


_________________________________________________________________________________________________________________________________
## Understanding the Datasets: The following code shows that there are no words that are in common between the corpus and the testing set

In [8]:
import os

# --- Load Corpus Words into a Set ---
KAGGLE_CORPUS_PATH = "/kaggle/input/corpus2/corpus.txt"
corpus_words = set()
try:
    with open(KAGGLE_CORPUS_PATH, 'r') as f:
        for word in f:
            corpus_words.add(word.strip().upper())
    print(f"Loaded {len(corpus_words)} unique words from corpus.txt")
except Exception as e:
    print(f"Error loading corpus: {e}")

# --- Load Test Words into a Set ---
KAGGLE_TEST_SET_PATH = "/kaggle/input/corpus2/test.txt"
test_words = set()
try:
    with open(KAGGLE_TEST_SET_PATH, 'r') as f:
        for word in f:
            test_words.add(word.strip().upper())
    print(f"Loaded {len(test_words)} unique words from test_set.txt")
except Exception as e:
    print(f"Error loading test set: {e}")

# --- Calculate and Print Overlap ---
if test_words and corpus_words:
    overlap = test_words.intersection(corpus_words)
    overlap_percentage = (len(overlap) / len(test_words)) * 100
    
    print("\n--- 📊 Diagnostic Results ---")
    print(f"Total Test Words:     {len(test_words)}")
    print(f"Total Corpus Words:   {len(corpus_words)}")
    print(f"Words in BOTH files:  {len(overlap)}")
    print(f"Overlap Percentage:   {overlap_percentage:.2f}%")
    
    if overlap_percentage == 0.0:
        print("\n**CRITICAL FINDING:** Your test set has 0% overlap with your training corpus.")
        print("This means the corpus filtering logic will *never* be used during evaluation.")
        print("The score you are seeing is purely from your bigram fallback model.")
    elif overlap_percentage < 100.0:
        print(f"\n**FINDING:** Only {overlap_percentage:.2f}% of your test words are in the corpus.")
        print("This means your filter is only working for a fraction of the games.")
    else:
        print("\n**FINDING:** 100% of test words are in the corpus.")
        print("This is good! It means the problem is a logic bug in Cell 4.")

Loaded 49398 unique words from corpus.txt
Loaded 2000 unique words from test_set.txt

--- 📊 Diagnostic Results ---
Total Test Words:     2000
Total Corpus Words:   49398
Words in BOTH files:  0
Overlap Percentage:   0.00%

**CRITICAL FINDING:** Your test set has 0% overlap with your training corpus.
This means the corpus filtering logic will *never* be used during evaluation.
The score you are seeing is purely from your bigram fallback model.


In [4]:
def train_hmm_models(words_by_length):
    """
    Trains positional Unigram, Bigram, and Trigram models for each word length.
    """
    hmm_models = {}
    
    for length, words in words_by_length.items():
        if length == 0:
            continue
            
        model = {}
        
        # --- 1. Unigram Model (Always possible for length >= 1) ---
        unigram_counts = np.ones((length, 26))
        for word in words:
            try:
                for pos, char in enumerate(word):
                    unigram_counts[pos, CHAR_TO_INDEX[char]] += 1
            except KeyError:
                continue
        model['unigram'] = unigram_counts / unigram_counts.sum(axis=1, keepdims=True)

        # --- 2. Bigram Model (Only possible for length >= 2) ---
        if length >= 2:
            bigram_counts = np.ones((length - 1, 26, 26))
            for word in words:
                try:
                    for pos in range(length - 1):
                        prev_char_idx = CHAR_TO_INDEX[word[pos]]
                        curr_char_idx = CHAR_TO_INDEX[word[pos + 1]]
                        bigram_counts[pos, prev_char_idx, curr_char_idx] += 1
                except KeyError:
                    continue
            model['bigram'] = bigram_counts / bigram_counts.sum(axis=2, keepdims=True)

        # --- 3. Trigram Model (Only possible for length >= 3) ---
        if length >= 3:
            trigram_counts = np.ones((length - 2, 26, 26, 26))
            for word in words:
                try:
                    for pos in range(length - 2):
                        prev_prev_char_idx = CHAR_TO_INDEX[word[pos]]
                        prev_char_idx = CHAR_TO_INDEX[word[pos + 1]]
                        curr_char_idx = CHAR_TO_INDEX[word[pos + 2]]
                        trigram_counts[pos, prev_prev_char_idx, prev_char_idx, curr_char_idx] += 1
                except KeyError:
                    continue
            model['trigram'] = trigram_counts / trigram_counts.sum(axis=3, keepdims=True)
        
        # Store the trained models
        hmm_models[length] = model
        
    print(f"Training complete. {len(hmm_models)} (U, B, T) models trained.")
    return hmm_models

hmm_models = train_hmm_models(words_by_length)

# --- Example: Check 'TH' -> 'E' transition for 5-letter words ---
if 5 in hmm_models:
    t_idx = CHAR_TO_INDEX['T']
    h_idx = CHAR_TO_INDEX['H']
    e_idx = CHAR_TO_INDEX['E']
    
    # Get P(Letter_2 | Letter_0='T', Letter_1='H')
    if 'trigram' in hmm_models[5]:
        prob_th_e = hmm_models[5]['trigram'][0, t_idx, h_idx, e_idx]
        print("\n--- Trigram Model Sanity Check (5-letter words) ---")
        print(f"  P(Letter at pos 2 = 'E' | ...pos 0 = 'T', pos 1 = 'H') = {prob_th_e:.4f}")
    else:
        print("Trigram model for length 5 not available (this shouldn't happen).")

Training complete. 24 (U, B, T) models trained.

--- Trigram Model Sanity Check (5-letter words) ---
  P(Letter at pos 2 = 'E' | ...pos 0 = 'T', pos 1 = 'H') = 0.0750


## Formulating the HMM (from scratch)

____________________________________________________________________________________________________________________

In [5]:
class HMMOracle:
    def __init__(self, models):
        self.models = models
        print("HMMOracle (Trigram w/ Backoff) initialized.")

    def _get_prob_with_backoff(self, model, pos, pattern, char_index):
        """
        Gets the probability of a character at a specific position,
        using the best available n-gram model (Trigram -> Bigram -> Unigram).
        """
        
        # --- Try Trigram ---
        # P(char | char_at_pos-2, char_at_pos-1)
        if pos >= 2 and pattern[pos-2] != '_' and pattern[pos-1] != '_':
            prev_prev_idx = CHAR_TO_INDEX[pattern[pos-2]]
            prev_idx = CHAR_TO_INDEX[pattern[pos-1]]
            # trigram_probs[pos-2, char_{i-2}, char_{i-1}, char_i]
            return model['trigram'][pos-2, prev_prev_idx, prev_idx, char_index]
            
        # --- Try Bigram ---
        # P(char | char_at_pos-1)
        if pos >= 1 and pattern[pos-1] != '_':
            prev_idx = CHAR_TO_INDEX[pattern[pos-1]]
            # bigram_probs[pos-1, char_{i-1}, char_i]
            return model['bigram'][pos-1, prev_idx, char_index]
            
        # --- Fallback to Unigram ---
        # P(char_i)
        return model['unigram'][pos, char_index]
        

    def get_letter_probabilities(self, pattern, guessed_letters):
        """
        Estimates the probability of each remaining letter appearing
        in one of the blank spots, using trigram backoff logic.
        """
        word_length = len(pattern)
        
        if word_length not in self.models:
            # Fallback for models we couldn't train (e.g., length 1)
            unguessed = [c for c in ALPHABET if c not in guessed_letters]
            if not unguessed: return {}
            prob = 1.0 / len(unguessed)
            return {char: prob for char in unguessed}
            
        model = self.models[word_length]
        
        blank_indices = [i for i, char in enumerate(pattern) if char == '_']
        unguessed_chars = [c for c in ALPHABET if c not in guessed_letters]
        
        if not blank_indices or not unguessed_chars:
            return {}
            
        final_scores = {char: 0.0 for char in unguessed_chars}
        
        # 3. Iterate over each blank and score each candidate letter
        for blank_pos in blank_indices:
            for char in unguessed_chars:
                char_index = CHAR_TO_INDEX[char]
                
                # --- P(Letter | Left_Neighbors) ---
                # Get the probability of this char given its left context
                left_prob = self._get_prob_with_backoff(model, blank_pos, pattern, char_index)
                
                # --- P(Right_Neighbors | Letter) ---
                # Now, find how well this char "predicts" its right-side context
                right_prob = 1.0
                
                # Check for known letter at pos+1
                if blank_pos + 1 < word_length and pattern[blank_pos+1] != '_':
                    next_idx = CHAR_TO_INDEX[pattern[blank_pos+1]]
                    # P(char_at_pos+1 | char_at_pos)
                    right_prob *= model['bigram'][blank_pos, char_index, next_idx]
                
                # Check for known letter at pos+2
                if blank_pos + 2 < word_length and pattern[blank_pos+1] != '_' and pattern[blank_pos+2] != '_':
                    next_idx = CHAR_TO_INDEX[pattern[blank_pos+1]]
                    next_next_idx = CHAR_TO_INDEX[pattern[blank_pos+2]]
                    # P(char_at_pos+2 | char_at_pos, char_at_pos+1)
                    right_prob *= model['trigram'][blank_pos, char_index, next_idx, next_next_idx]
                
                # This score represents how well this char "fits" in this blank
                prob_at_this_pos = left_prob * right_prob
                final_scores[char] += prob_at_this_pos

        # 4. Normalize scores to create a probability distribution
        total_score = sum(final_scores.values())
        
        if total_score == 0.0:
            # Fallback: No letter fits, return uniform prob
            prob = 1.0 / len(unguessed_chars)
            return {char: prob for char in unguessed_chars}
            
        normalized_probs = {
            char: score / total_score
            for char, score in final_scores.items()
        }
        
        return normalized_probs

# --- Initialize the new oracle ---
if 'hmm_models' in locals():
    oracle = HMMOracle(hmm_models)
else:
    print("Error: 'hmm_models' not initialized. Please re-run Cell 3.")

HMMOracle (Trigram w/ Backoff) initialized.


## Example:  Three words: "S____", "_PPLE", __A_I_G"

______________________________________________________________________________________________________________________________

In [6]:
# --- Example 1: A common 5-letter word start ---
pattern1 = "S____"
guessed1 = {'S'}
probs1 = oracle.get_letter_probabilities(pattern1, guessed1)

print(f"--- Probs for '{pattern1}' (guessed: {guessed1}) ---")
# Sort by probability, descending
for char, prob in sorted(probs1.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"  P({char}): {prob:.4f}")


# --- Example 2: The 'APPLE' example from the prompt ---
pattern2 = "_PPLE"
guessed2 = {'P', 'L', 'E'}
probs2 = oracle.get_letter_probabilities(pattern2, guessed2)

print(f"\n--- Probs for '{pattern2}' (guessed: {guessed2}) ---")
for char, prob in sorted(probs2.items(), key=lambda item: item[1], reverse=True)[:5]:
    print(f"  P({char}): {prob:.4f}")

    
# --- Example 3: More complex pattern ---
pattern3 = "__A_I_G"
guessed3 = {'A', 'I', 'G'}
probs3 = oracle.get_letter_probabilities(pattern3, guessed3)

print(f"\n--- Probs for '{pattern3}' (guessed: {guessed3}) ---")
for char, prob in sorted(probs3.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"  P({char}): {prob:.4f}")

--- Probs for 'S____' (guessed: {'S'}) ---
  P(E): 0.1110
  P(A): 0.1020
  P(T): 0.0788
  P(I): 0.0689
  P(O): 0.0680
  P(N): 0.0642
  P(L): 0.0616
  P(R): 0.0597
  P(H): 0.0460
  P(C): 0.0452

--- Probs for '_PPLE' (guessed: {'P', 'L', 'E'}) ---
  P(S): 0.3186
  P(U): 0.1231
  P(A): 0.1220
  P(M): 0.0451
  P(H): 0.0417

--- Probs for '__A_I_G' (guessed: {'I', 'A', 'G'}) ---
  P(N): 0.1331
  P(S): 0.0988
  P(C): 0.0816
  P(P): 0.0694
  P(R): 0.0683
  P(T): 0.0681
  P(B): 0.0622
  P(L): 0.0589
  P(M): 0.0515
  P(D): 0.0482


In [11]:
import os
import random

KAGGLE_TEST_SET_PATH = "/kaggle/input/corpus2/test.txt"

def load_test_set(filename=KAGGLE_TEST_SET_PATH):
    """
    Loads the test set words from the specified Kaggle path.
    """
    # Check if the file exists
    if not os.path.exists(filename):
        print(f"--- ERROR ---")
        print(f"Test set file not found at: {filename}")
        print("Please verify the 'corpus2' folder name and the test set file name.")
        
        # Helper to debug: list contents of the directory
        print(f"\nListing contents of {os.path.dirname(filename)} ...")
        try:
            print(os.listdir(os.path.dirname(filename)))
        except Exception as e:
            print(f"Could not list directory: {e}")
        return None

    # File exists, proceed to load
    try:
        with open(filename, 'r') as f:
            test_words = [word.strip().upper() for word in f if word.strip()]
        
        print(f"Test set loaded successfully from {filename}.")
        print(f"Found {len(test_words)} test words.")
        return test_words
        
    except Exception as e:
        print(f"An error occurred while reading the test file: {e}")
        return None

def play_game(secret_word, oracle, max_lives=6):
    """
    Simulates a single game of Hangman using a greedy agent
    that trusts the HMMOracle.
    """
    word_length = len(secret_word)
    pattern = "_" * word_length
    guessed_letters = set()
    
    wrong_guesses = 0
    repeated_guesses = 0 # Will be 0 with our current oracle
    lives = max_lives
    
    while lives > 0 and "_" in pattern:
        probs = oracle.get_letter_probabilities(pattern, guessed_letters)
        
        if not probs:
            break # Game is lost
            
        # 3. Greedy Agent: Pick the best letter
        guess = max(probs, key=probs.get)
        
        
        guessed_letters.add(guess)
        
        if guess in secret_word:
            # Correct guess: update pattern
            new_pattern = list(pattern)
            for i in range(word_length):
                if secret_word[i] == guess:
                    new_pattern[i] = guess
            pattern = "".join(new_pattern)
        else:
            # Wrong guess
            wrong_guesses += 1
            lives -= 1
            
    # 5. Return game results
    game_won = "_" not in pattern
    return game_won, wrong_guesses, repeated_guesses

# --- Main Evaluation Loop ---

# Load the test words
test_words = load_test_set()

if test_words and oracle:
    NUM_GAMES = 2000 # As specified in the PDF 
    MAX_LIVES = 6      # As specified in the PDF 
    
    total_games_won = 0
    total_wrong_guesses = 0
    total_repeated_guesses = 0
    
    print(f"\n--- Starting Evaluation: Playing {NUM_GAMES} games ---")

    if not test_words:
        print("Cannot run evaluation: test_words list is empty.")
    else:
        test_set_size = len(test_words)
        
        for i in range(NUM_GAMES):
           
            secret_word = random.choice(test_words)
            
            if len(secret_word) not in oracle.models:
                # print(f"Skipping word '{secret_word}': No model for length {len(secret_word)}")
                continue

            won, wrongs, repeats = play_game(secret_word, oracle, MAX_LIVES)
            
            if won:
                total_games_won += 1
            total_wrong_guesses += wrongs
            total_repeated_guesses += repeats
            
            if (i + 1) % 200 == 0:
                print(f"  ... completed {i + 1} / {NUM_GAMES} games")

        print("--- Evaluation Complete ---")
        success_rate = total_games_won / NUM_GAMES
        avg_wrong_guesses = total_wrong_guesses / NUM_GAMES
        avg_repeated_guesses = total_repeated_guesses / NUM_GAMES
        
        # 2. Calculate Final Score using the formula 
        # Final Score = (Success Rate * 2000) - (Total Wrong Guesses * 5) - (Total Repeated Guesses * 2)
        
        # (Success Rate * 2000) is just total_games_won
        score_from_wins = success_rate * 2000 
        penalty_from_wrongs = total_wrong_guesses * 5
        penalty_from_repeats = total_repeated_guesses * 2
        
        final_score = score_from_wins - penalty_from_wrongs - penalty_from_repeats
        
        # 3. Print Results
        print("\n--- 📊 Final Results ---")
        print(f"**Final Score:** {final_score:.2f}")
        print("------------------------------")
        print(f"Total Games Played:     {NUM_GAMES}")
        print(f"Total Games Won:        {total_games_won} ({success_rate * 100:.2f}%)")
        print("------------------------------")
        print(f"Total Wrong Guesses:    {total_wrong_guesses} (Avg: {avg_wrong_guesses:.2f} per game)")
        print(f"Total Repeated Guesses: {total_repeated_guesses} (Avg: {avg_repeated_guesses:.2f} per game)")
        print("------------------------------")
        print(f"Score from Wins:        + {score_from_wins:.2f}")
        print(f"Penalty from Wrongs:    - {penalty_from_wrongs:.2f}")
        print(f"Penalty from Repeats:   - {penalty_from_repeats:.2f}")
        
else:
    print("\nEvaluation not run. Could not load test set or oracle is not initialized.")

Test set loaded successfully from /kaggle/input/corpus2/test.txt.
Found 2000 test words.

--- Starting Evaluation: Playing 2000 games ---
  ... completed 200 / 2000 games
  ... completed 400 / 2000 games
  ... completed 600 / 2000 games
  ... completed 800 / 2000 games
  ... completed 1000 / 2000 games
  ... completed 1200 / 2000 games
  ... completed 1400 / 2000 games
  ... completed 1600 / 2000 games
  ... completed 1800 / 2000 games
  ... completed 2000 / 2000 games
--- Evaluation Complete ---

--- 📊 Final Results ---
**Final Score:** -47859.00
------------------------------
Total Games Played:     2000
Total Games Won:        796 (39.80%)
------------------------------
Total Wrong Guesses:    9731 (Avg: 4.87 per game)
Total Repeated Guesses: 0 (Avg: 0.00 per game)
------------------------------
Score from Wins:        + 796.00
Penalty from Wrongs:    - 48655.00
Penalty from Repeats:   - 0.00


## Members:

#### Prateek Meher K

#### Kushal Kumar G

#### Mahima R Shetty

#### Jyotsana S

______________________________________________________________________________________________________________________________